# Early game running start

### Source - https://medium.com/@rqaiserr/how-to-convert-pdfs-into-searchable-key-words-with-python-85aab86c544f

In [18]:
import pandas as pd
import numpy as np
import regex as re
import requests
import nltk
import PyPDF2 
#import textract
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
#nltk.download('punkt') # necessary jaxzz to tokenize
# for downloading pdf's

## Step 1: Clean Main dataset and Download State PDF Contracts Remote->Local

In [19]:
# Import raw dataset from kaggle
data = pd.read_csv('../raw/contracts.csv', encoding='utf-8')
# Change column headers to utilize underlines instead of whitespace.
data.columns = data.columns.str.replace(' ', '_')
# Use list comprehension to make all headers lowercase as well.
data.columns = [x.lower() for x in data.columns]
# Change contract_url header to say just that...
data.rename(columns={'contract_pdf': 'contract_url'}, inplace=True)
# Output first 5 rows (head)
data.head()

C:\Users\kubickil\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,purchase_order_description,purchase_order_(contract)_number,revision_number,specification_number,contract_type,start_date,end_date,approval_date,department,vendor_name,vendor_id,address_1,address_2,city,state,zip,award_amount,procurement_type,contract_url
0,JANITORIAL SUPPLIES,T27497,0,B74851001,NaN,1997-12-01T00:00:00,1999-11-30T00:00:00,NaN,NaN,"IMM.M.M., INC",15193295T,5262 S KOLMAR AVE,NaN,CHICAGO,IL,60632-3750,0.0,NaN,NaN
1,DEMOLITION,E011442,0,E968020030,NaN,NaN,NaN,NaN,DEPARTMENT OF BUILDINGS,MIDWEST WRECKING COMPANY 01,14878414T,1950 W HUBBARD ST,NaN,CHICAGO,IL,60622,17500.0,NaN,NaN
2,JANITORIAL SUPPLIES,T27500,0,B74851001,NaN,1997-12-01T00:00:00,1999-11-30T00:00:00,NaN,NaN,"CHICAGO UNITED INDUSTRIES, LIMITED",22085024V,53 W JACKSON BLVD # 1450,NaN,CHICAGO,IL,60604-3806,0.0,NaN,NaN
3,MASTER AGREEMENT FOR DEMOLITION SERVICES,C02163,0,B89683203,NaN,NaN,NaN,NaN,DEPARTMENT OF BUILDINGS,"DEMOLITION & DEVELOPMENT, LIMITED.",22414299V,P.0. BOX 10263,NaN,CHICAGO,IL,60610,7000.0,NaN,NaN
4,MASTER AGREEMENT FOR DEMOLITION SERVICES,C02106,0,B89683203,NaN,NaN,NaN,NaN,DEPARTMENT OF BUILDINGS,"DEMOLITION & DEVELOPMENT, LIMITED.",22414299V,P.0. BOX 10263,NaN,CHICAGO,IL,60610,8200.0,NaN,NaN


In [20]:
# How many rows? (before filter)
len(data.index)

130798

In [21]:
# Filter and create a df with only contracts that have an attached url
data['contract_url'].replace('', np.nan, inplace=True)
data.dropna(subset=['contract_url'], inplace=True)
# How many rows? (after filter should  be 45,775)
len(data.index)

45775

In [22]:
# See where were at again...
data.head()

,purchase_order_description,purchase_order_(contract)_number,revision_number,specification_number,contract_type,start_date,end_date,approval_date,department,vendor_name,vendor_id,address_1,address_2,city,state,zip,award_amount,procurement_type,contract_url
50825,FY 04 HOMELESS SERVICES,2155,0,11655,DELEGATE AGENCY,2003-07-01T00:00:00,2005-06-30T00:00:00,2003-06-26T00:00:00,DEPARTMENT OF HUMAN SERVICES,LAKE FRONT SRO CORP.,3970415Z,4946 N SHERIDAN RD,NaN,CHICAGO,IL,60640,204000.0,RFP,{'url': 'http://ecm.cityofchicago.org/eSMARTCo...
50826,Second Half of Cultural Outreach Program 2012,26792,0,110510,DELEGATE AGENCY,2012-07-01T00:00:00,2013-02-15T00:00:00,2012-08-30T00:00:00,DEPARTMENT OF CULTURAL AFFAIRS,CHANGING WORLDS,91371340Z,329 W 18TH STREET EFT ST,NaN,CHICAGO,IL,60616,6000.0,NaN,{'url': 'http://ecm.cityofchicago.org/eSMARTCo...
50827,Open Space - 4228 W. Ogden,29130,0,121371,COMPTROLLER-OTHER,2013-10-16T00:00:00,2014-12-31T00:00:00,2013-11-08T00:00:00,DEPT OF COMMUNITY DEVELOPMENT,OPENLANDS,93410144P,AMERICAN NATL BK #784699,33 N LA SALLE ST,CHICAGO,IL,60690,192384.1,NaN,{'url': 'http://ecm.cityofchicago.org/eSMARTCo...
50828,DUMP TRAILERS,9982,0,35680,VEHICLES/HEAVY EQUIPMENT (CAPITAL),2005-11-01T00:00:00,2008-10-31T00:00:00,2005-11-08T00:00:00,DEPT OF FLEET MGMT,R G SMITH EQUIPMENT CO.,31303247L,622 NORTHWEST HWY,NaN,DES PLAINES,IL,60018,809580.0,BID,{'url': 'http://ecm.cityofchicago.org/eSMARTCo...
50829,"CAB/CHASSIS WITH UTILITY BODY BOOM, 54' BUCKET...",T24436,0,B10705307,Term Agreement,2002-07-01T00:00:00,2004-06-30T00:00:00,2002-06-21T00:00:00,DEPT OF STREETS & SANITATION,MID-AMERICA TRUCK&EQUIP CO INC,33668302P,625 SOUTH RTE 83,NaN,ELMHURST,IL,60126,0.0,NaN,{'url': 'http://ecm.cityofchicago.org/eSMARTCo...


In [23]:
# Now use regex to filter and change column to have JUST the url itself:
# Just grab group 1
remove_junk = re.compile(r"{'url':\s'(.*)'}", re.IGNORECASE)
data['contract_url'] = data['contract_url'].apply(lambda x: re.search(remove_junk, x).group(1))

In [33]:
data.loc[data['department'] == 'CHICAGO DEPARTMENT OF TRANSPORTATION']#['contract_type'].value_counts()

,purchase_order_description,purchase_order_(contract)_number,revision_number,specification_number,contract_type,start_date,end_date,approval_date,department,vendor_name,vendor_id,address_1,address_2,city,state,zip,award_amount,procurement_type,contract_url
50830,Arterial Street ADA Ramp Construction - Far So...,22515,5,80000,CONSTRUCTION-GENERAL,NaN,NaN,2013-03-06T00:00:00,CHICAGO DEPARTMENT OF TRANSPORTATION,G & V CONSTRUCTION CO INC,90506630P,371 S EVERGREEN STREET EFT,NaN,BENSENVILLE,IL,60106,50015.5,BID,http://ecm.cityofchicago.org/eSMARTContracts/s...
50837,E. 130th Street/S. Brainard Ave./S. Torrence A...,23954,56,24508A,CONSTRUCTION-GENERAL,NaN,NaN,2013-09-05T00:00:00,CHICAGO DEPARTMENT OF TRANSPORTATION,WALSH CONSTRUCTION COMPANY OF ILL,15792481T,929 W ADAMS STREET EFT,NaN,CHICAGO,IL,60607,50000.0,BID,http://ecm.cityofchicago.org/eSMARTContracts/s...
50838,PN84P012021A NEW STREET,P012021A,4,PU01390101A,CONSTRUCTION,NaN,NaN,2003-04-24T00:00:00,CHICAGO DEPARTMENT OF TRANSPORTATION,"HAMILTON CONSTRUCTION, INC.",15241383H,342 N LAGRANGE RD STE 352,NaN,FRANKFORT,IL,60449,149150.4,NaN,http://ecm.cityofchicago.org/eSMARTContracts/s...
50845,Target Market Task Order Professional Design E...,14292,0,54805,ARCH/ENGINEERING,2007-04-01T00:00:00,2010-03-31T00:00:00,2007-05-18T00:00:00,CHICAGO DEPARTMENT OF TRANSPORTATION,"PRIMERA ENGINEERS, LIMITED",22638829H,100 S WACKER DR # 700,NaN,CHICAGO,IL,60606,1000000.0,RFP,http://ecm.cityofchicago.org/eSMARTContracts/s...
50848,CHICAGO GREENSTREETS TREE PLANTING INITIATIVE ...,28484,0,94974A,CONSTRUCTION-GENERAL,NaN,NaN,2013-12-20T00:00:00,CHICAGO DEPARTMENT OF TRANSPORTATION,SEVEN-D CONSTRUCTION CO.,93201616L,2000 W 43RD STREET EFT ST,NaN,CHICAGO,IL,60609,790400.0,BID,http://ecm.cityofchicago.org/eSMARTContracts/s...
50857,ADA Ramp Program - 2010 South Area,22189,2,79284,CONSTRUCTION-LARGE $3MILLIONorABOVE,2010-06-18T00:00:00,2013-12-31T00:00:00,2012-11-08T00:00:00,CHICAGO DEPARTMENT OF TRANSPORTATION,"SUMIT CONSTRUCTION CO., INC.",21973746A,4150 W WRIGHTWOOD AVE FL 1ST,NaN,CHICAGO,IL,60639,0.0,BID,http://ecm.cityofchicago.org/eSMARTContracts/s...
50861,SOUTH LAKE SHORE DRIVE - JACKSON PARK SECTION ...,2606,0,PB14410201,CONSTRUCTION-LARGE $3MILLIONorABOVE,NaN,NaN,2003-09-12T00:00:00,CHICAGO DEPARTMENT OF TRANSPORTATION,WALSH CONSTRUCTION COMPANY OF ILL,15792481T,929 W ADAMS STREET EFT,NaN,CHICAGO,IL,60607,18047962.9,BID,http://ecm.cityofchicago.org/eSMARTContracts/s...
50862,Temporary Easement for 3110 E. 79th St.,16591,0,62500,COMPTROLLER-OTHER,2007-08-14T00:00:00,2008-12-31T00:00:00,2008-01-08T00:00:00,CHICAGO DEPARTMENT OF TRANSPORTATION,WHEATLAND TITLE GUARANTY CO.,89964027V,105 W VETERANS PKWY,NaN,YORKVILLE,IL,60560,52900.0,NaN,http://ecm.cityofchicago.org/eSMARTContracts/s...
50874,NORTH LAKE SHORE DRIVE PAVEMENT OVERLAY - FOST...,17957,0,58625A,CONSTRUCTION-LARGE $3MILLIONorABOVE,NaN,NaN,2008-08-11T00:00:00,CHICAGO DEPARTMENT OF TRANSPORTATION,ARROW ROAD CONSTRUCTION CO.,25942130T,3401 S BUSSE RD,NaN,MOUNT PROSPECT,IL,60056,6046565.9,BID,http://ecm.cityofchicago.org/eSMARTContracts/s...
50875,Professional Construction Engineering Services...,14758,1,53021,ARCH/ENGINEERING,2007-05-01T00:00:00,2014-04-30T00:00:00,2012-08-27T00:00:00,CHICAGO DEPARTMENT OF TRANSPORTATION,COLLINS ENGINEERS INC,35882066V,123 N WACKER DRIVE EFT,NaN,CHICAGO,IL,60606,4000000.0,NaN,http://ecm.cityofchicago.org/eSMARTContracts/s...


In [16]:
# At this point for each row we'll download the pdf and store the pdf locally.
    
def download_pdf(row):
    url=row['contract_url']
    response = requests.get(url) # We'll grab the response.text (html output of page), grab REAL pdf link and download.
    # Use regex to pull the link out....Let's hope all chicago contracts follow same html format ;)
    pdf_regex = re.compile(r'<iframe src="(.*)"\sname=', re.IGNORECASE) # find link; this regex should be standard on all state pages.
    new_link = re.search(pdf_regex, response.text).group(1) # REAL pdf link
    r = requests.get(new_link, allow_redirects=True, stream=True)
    with open('../raw/chicago_pdfs/' + str(row['specification_number']) + '-' + str(row['vendor_id']) + '.pdf', 'wb') as f:
        f.write(r.content)

# Download pdf for each row using its contract_url
data.apply(download_pdf, axis=1)



# VERIFIED EXTRACTION CODE TO DOWNLOAD PDF FROM STATE SITES (pain in the butt, took me way too long...)

# url = 'http://ecm.cityofchicago.org/eSMARTContracts/service/DPSWebDocumentViewer?sid=ESMART&id={2488393F-CCF9-476E-808A-9FBF3C25E0D6}'
# response = requests.get(url) # We'll grab the response.text (html output of page), grab real pdf link and download.
# # Use regex to pull the link out....Let's hope all chicago contracts follow same html format ;)
# pdf_regex = re.compile(r'<iframe src="(.*)"\sname=', re.IGNORECASE) # find link; this regex should be standard on all state pages.
# new_link = re.search(pdf_regex, response.text).group(1)
# r = requests.get(new_link, allow_redirects=True, stream=True)
# with open('test.pdf', 'wb') as f:
#     f.write(r.content)

KeyboardInterrupt: 

In [ ]:
# This  function says for each row we have an associated pddf, extract text from it. (see what happens!)
def pdf_to_text(row): 
    # Grab what the filename should be
    filename = '../raw/chicago_pdfs/' + str(row['specification_number']) + '-' + str(row['vendor_id']) + '.pdf'
        
# For each row take text from downloaded pdf associated file/delete it?
data['text_list'] = data.apply(pdf_to_text, axis=1)

## STEP 2: Local_PDF -> List of strings representing OCR output

In [17]:
# https://gitlab.gnome.org/World/OpenPaperwork/pyocr
from wand.image import Image
from PIL import Image as PI
import pyocr
import pyocr.builders
import io
import sys
# Get english language for OCR
from pyocr import tesseract as tool
tool = pyocr.get_available_tools()[0] # this didnt work use tesseract
lang = tool.get_available_languages()[1]
req_image = []
final_text = []
# Convert a pdf to jpg for OCR
image_pdf = Image(filename="../raw/chicago_pdfs/ocr_test.pdf", resolution=300)
image_jpeg = image_pdf.convert('jpeg')
# Append all pdf converted jpgs to a blob object
for img in image_jpeg.sequence:
    img_page = Image(image=img)
    req_image.append(img_page.make_blob('jpeg'))


Unsupported version [0.0.0]


In [2]:
# FINALLY run tesseract-OCR over our blob jpg images
for img in req_image: 
    txt = tool.image_to_string(
        PI.open(io.BytesIO(img)),
        lang='eng',#lang
        builder=pyocr.builders.TextBuilder()
    )
    final_text.append(txt)

In [6]:
final_text

['Modification Summary Report\n\nContract (PO) Number: 124572\n\nModification Revision Number: 8\n\nSpecification Number: B19182915\n\nName of Contractor: ORACLE CORPORATION\n\nCity Department: DEPARTMENT OF POLICE\n\nTitle of Contract: COMPUTER SOFTWARE CONSULTING\n\nMod Reg Number Mod Reason . Description\nAMENDMENT TO PO #\nT24572/ORACLE FOR $10\n38378 DOLLAR AMOUNT CHANGE = TL LION LIMIT INCREASE AND\n24 MONTH EXTENSION\n\n \n\nTerm of Contract: Start Date: 5/31/2001\n\nEnd Date: 14/31/2010\n\nProcurement Services Contact Person: CHARLITA FAIN\n\nPlease refer to the DPS website for Contact information under "Doing Business With The City".\n\n \n\nVendor Number: 1031093\n\nSubmission Date:',
 '_ COMPLETE\nAMENDMENT |\nFOR',
 'Specification No.: _ B-191-82915\nContract (P.O.) No.: T24572 ("Agreement")\n\nAmendment No.: 8\nVendor No.: 1031093\nAMENDMENT\n\nThis amendment (“Amendment”) is made and entered into effective as of the 1st day of J uly, 2008 by and -\nbetween the CITY OF CHI

In [9]:
import regex as re
procure = re.compile(r'procurement\splan', re.IGNORECASE)
test = bool(procure.search(final_text[0]))# This pulls first element of the list 0th element, aka the big string
test

False

In [ ]:
# First objective: Create column that contains text from pdf->text (tesseract?)
# (Or better option than tesseract for python if exists...)

# Late game

In [ ]:
# Second objective: Make a column utilizing web scraping on 

In [ ]:
# Third objective: Create truth column that deduces whether or not a contract
# was ACCEPTED/DENIED utiilizing regex and looking at column created above.


In [ ]:
# Fourth objective: Create a list of prioritized vendors to imititate based off:
# -How many contracts they've acquired
# -Value of contracts (ie: give higher weight/importance of imitation to
    # high-paying contract obtainers)

In [ ]:
# Once curated dataset created, upload to Kaggle as a kernel and call TIDY_contract_data.csv/xlsx


In [ ]:
# Create a text classifier that finds and verifies a piece ofo text exists in
# all contracts (ie: a standard or substandard that is followed but sometimes  forgotten)

In [ ]:
# OR create a feature bayesian inference to discern acceptance or denial of contract